# Movielens dataset

David Galera

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
ratings = pd.read_csv("/content/drive/MyDrive/Advanced ML/ratings.csv")
print(ratings.shape)
ratings.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movies = pd.read_csv("/content/drive/MyDrive/Advanced ML/movies.csv")
print(movies.shape)
movies.head()

(9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Primer calculem el *rating* promig de cada pel·lícula:

In [5]:
ratings_mean=ratings.groupby('movieId')['rating'].mean().rename('Mean_rating').reset_index()
ratings_mean.head()

,movieId,Mean_rating
0,1,3.920930
1,2,3.431818
2,3,3.259615
3,4,2.357143
4,5,3.071429


Calculem la mitjana amortida del *rating* de cada pel·lícula amb alfa=5:

In [7]:
alfa=5

Primer calculem el sumatori de tots els *ratings* per a cada pel·lícula i posteriorment el mateix sumatori + alpha vegades el promig total de *ratings* (el dividend de l'equació de la mitjana amortida):

In [8]:
ratings_sum=ratings.groupby('movieId')['rating'].sum().rename('sum_rating').reset_index()
ratings_sum['sum_rating_factor']=ratings_sum['sum_rating']+alfa*(ratings['rating'].mean())
ratings_sum.head()

,movieId,sum_rating,sum_rating_factor
0,1,843.0,860.507785
1,2,377.5,395.007785
2,3,169.5,187.007785
3,4,16.5,34.007785
4,5,150.5,168.007785


A continuació calculem el número de vegades que s'ha puntuat cada pel·lícula i repetim la operació sumant-hi alfa (el divisor de l'equació de la mitjana amortida):

In [9]:
ratings_count=ratings.groupby('movieId')['rating'].count().rename('count_rating').reset_index()
ratings_count['count_rating_factor']=ratings_count['count_rating']+alfa
ratings_count.head()

,movieId,count_rating,count_rating_factor
0,1,215,220
1,2,110,115
2,3,52,57
3,4,7,12
4,5,49,54


Procedim a calcular la mitjana amortida: 

*sum_rating_factor*/*count_rating_factor*

In [13]:
ratings_damped=pd.merge(ratings_sum,ratings_count,on=['movieId'],how='left')
ratings_damped['Damped_mean_rating']=ratings_damped['sum_rating_factor']/ratings_damped['count_rating_factor']
ratings_damped.head()

,movieId,sum_rating,sum_rating_factor,count_rating,count_rating_factor,Damped_mean_rating
0,1,843.0,860.507785,215,220,3.911399
1,2,377.5,395.007785,110,115,3.434850
2,3,169.5,187.007785,52,57,3.280838
3,4,16.5,34.007785,7,12,2.833982
4,5,150.5,168.007785,49,54,3.111255


Fusionem el DataFrame de la mitjana normal amb el de la mitjana amortida.

In [15]:
ratings_mean_dampmean=pd.merge(ratings_mean,ratings_damped[['movieId','Damped_mean_rating']],on=['movieId'],how='left')
ratings_mean_dampmean.head()

,movieId,Mean_rating,Damped_mean_rating
0,1,3.920930,3.911399
1,2,3.431818,3.434850
2,3,3.259615,3.280838
3,4,2.357143,2.833982
4,5,3.071429,3.111255


Inserim el títol i gènere de les pel·lícules:

In [17]:
ratings_movies = ratings_mean_dampmean.merge(movies,on="movieId")
ratings_movies.head()

,movieId,Mean_rating,Damped_mean_rating,title,genres
0,1,3.920930,3.911399,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,3.431818,3.434850,Jumanji (1995),Adventure|Children|Fantasy
2,3,3.259615,3.280838,Grumpier Old Men (1995),Comedy|Romance
3,4,2.357143,2.833982,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,3.071429,3.111255,Father of the Bride Part II (1995),Comedy


Ordenem el dataset segons la mitjana amortida

In [21]:
ratings_movies.sort_values("Damped_mean_rating",ascending=False,inplace=True)
ratings_movies.head()

,movieId,Mean_rating,Damped_mean_rating,title,genres
277,318,4.429022,4.414620,"Shawshank Redemption, The (1994)",Crime|Drama
840,1104,4.475000,4.280311,"Streetcar Named Desire, A (1951)",Drama
9600,177593,4.750000,4.269830,"Three Billboards Outside Ebbing, Missouri (2017)",Crime|Drama
659,858,4.289062,4.269075,"Godfather, The (1972)",Crime|Drama
2224,2959,4.272936,4.255640,Fight Club (1999),Action|Crime|Drama|Thriller


Quines són les *N* millors pelicules segons mitjana amortida?

Imprimim els 5 primers títols.

In [22]:
N=5
for j in list(ratings_movies.head(N).title): print (j)

Shawshank Redemption, The (1994)
Streetcar Named Desire, A (1951)
Three Billboards Outside Ebbing, Missouri (2017)
Godfather, The (1972)
Fight Club (1999)
